In [35]:
import pandas as pd
import wandb
api = wandb.Api()
from consts import (
    cnn_group_data_extra_features, BATCHSIZE, all_collections
)

databases = ["raw","meters","exponential","all_collections","all_spaces"]
spaces = ["indoor","garage","outdoor"]

model_dict:dict = {}

# Project is specified by <entity/project-name>
for group in cnn_group_data_extra_features.keys():
    runs = api.runs(
    "santi-mbenites-university-of-lisbon/wifi-rssi-gradient-search",
    filters={"group":group}
    )
    print(f"{group} : {len(runs)}")
    for run in runs:
        model_dict[group] = model_dict.get(group,[])
        model_dict[group].append(run.name)


CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features : 3
CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features : 3
CNN_DATA_ANALYSIS_reto_grande_garage_extra_features : 3
CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features : 3
CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features : 3


In [36]:
display(model_dict)

{'CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features': ['reto_grande_outdoor_extra_features_run0_depth0_model4',
  'reto_grande_outdoor_extra_features_run1_depth1_model2',
  'reto_grande_outdoor_extra_features_run2_depth0_model3'],
 'CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features': ['reto_grande_indoor_extra_features_run0_depth1_model5',
  'reto_grande_indoor_extra_features_run1_depth4_model4',
  'reto_grande_indoor_extra_features_run2_depth4_model1'],
 'CNN_DATA_ANALYSIS_reto_grande_garage_extra_features': ['reto_grande_garage_extra_features_run0_depth3_model1',
  'reto_grande_garage_extra_features_run1_depth3_model4',
  'reto_grande_garage_extra_features_run2_depth2_model0'],
 'CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features': ['all_spaces_reto_grande_extra_features_run0_depth3_model5',
  'all_spaces_reto_grande_extra_features_run1_depth2_model7',
  'all_spaces_reto_grande_extra_features_run2_depth4_model0'],
 'CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_fea

In [37]:
import os
from model_evaluation import evaluate_model_on_collections
from feature_lists import DATASET_TO_FEATURE

os.environ["CUDA_VISIBLE_DEVICES"] = "5"

results = {}
for group in cnn_group_data_extra_features:
    results[group] = []


for current_group,all_runs in model_dict.items():

    for current_run in all_runs:
        current_group_data      = cnn_group_data_extra_features[current_group]
        current_model_storage   = current_group_data["model_storage"]
        current_database        = current_group_data["database"]
        current_model_path      = f"{current_model_storage}/{current_run}.pt"
        feature_list            = DATASET_TO_FEATURE[current_database]
        relative_coeficient     = current_group_data["relative_coeficient"]
        model_output_name       = current_run
        val_collection          = current_group_data["validation_collections"]

        for collections_name, current_collections in val_collection.items(): 
            
            if not os.path.exists(current_model_path):
                print(f"Model doesnt exist {model_output_name} with path {current_model_path}")
                continue
            
            try:
                mae, mse, rmse, n = evaluate_model_on_collections(
                    current_model_path, current_collections, current_database, BATCHSIZE, None, feature_list
                )
            except Exception as e:
                print(f"Failed Processing {model_output_name} with path {current_model_path}")
                print(e)
                continue
            mae_final = mae * relative_coeficient
            mse_final = mse * relative_coeficient
            rmse_final = rmse * relative_coeficient
            print(f"{model_output_name} | Collections : {collections_name} | N={n} | MAE={mae_final:.6f} | MSE={mse_final:.6f} | RMSE={rmse_final:.6f}")
            results[current_group].append((model_output_name, collections_name, mae_final, mse_final, rmse_final))




reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_collections_garage | N=103431 | MAE=6.279112 | MSE=1.942305 | RMSE=7.883765
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_collections_outdoor | N=220184 | MAE=5.898936 | MSE=1.903960 | RMSE=7.805557
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_collections_indoor | N=38036 | MAE=22.917395 | MSE=30.781176 | RMSE=31.384672
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_collections_garage_outdoor | N=323615 | MAE=6.020458 | MSE=1.916222 | RMSE=7.830652
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_collections_garage_indoor | N=141467 | MAE=10.752625 | MSE=9.696168 | RMSE=17.614692
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : all_collections_indoor_outdoor | N=258220 | MAE=8.405772 | MSE=6.157598 | RMSE=14.037205
reto_grande_outdoor_extra_features_run0_depth0_model4 | Collections : a

In [38]:
import pandas as pd
for group, rows in results.items():

    df = pd.DataFrame(rows, columns=["model","collections", "mae", "mse", "rmse"])
    print(f"GROUP : {group}")
    styled = df.style.background_gradient(cmap="RdYlGn_r")  # _r reverses so green=low, red=high
    display(styled)

GROUP : CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features


,model,collections,mae,mse,rmse
0,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_garage,6.279112,1.942305,7.883765
1,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_outdoor,5.898936,1.903960,7.805557
2,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_indoor,22.917395,30.781176,31.384672
3,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_garage_outdoor,6.020458,1.916222,7.830652
4,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_garage_indoor,10.752625,9.696168,17.614692
5,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_indoor_outdoor,8.405772,6.157598,14.037205
6,reto_grande_outdoor_extra_features_run0_depth0_model4,all_collections_garage_outdoor_indoor,7.797554,4.952038,12.588297
7,reto_grande_outdoor_extra_features_run1_depth1_model2,all_collections_garage,10.771908,5.590982,13.375778
8,reto_grande_outdoor_extra_features_run1_depth1_model2,all_collections_outdoor,7.450868,3.093445,9.949384
9,reto_grande_outdoor_extra_features_run1_depth1_model2,all_collections_indoor,19.928508,21.212971,26.054080


GROUP : CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features


,model,collections,mae,mse,rmse
0,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_garage,9.548969,4.247868,11.658978
1,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_outdoor,9.528844,4.228463,11.632317
2,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_indoor,9.343408,5.821923,13.649233
3,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_garage_outdoor,9.535276,4.234665,11.640845
4,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_garage_indoor,9.493700,4.671082,12.225981
5,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_indoor_outdoor,9.501529,4.463181,11.950807
6,reto_grande_indoor_extra_features_run0_depth1_model5,all_collections_garage_outdoor_indoor,9.515097,4.401602,11.868077
7,reto_grande_indoor_extra_features_run1_depth4_model4,all_collections_garage,9.549054,4.247961,11.659106
8,reto_grande_indoor_extra_features_run1_depth4_model4,all_collections_outdoor,9.528912,4.228529,11.632408
9,reto_grande_indoor_extra_features_run1_depth4_model4,all_collections_indoor,9.343318,5.821776,13.649060


GROUP : CNN_DATA_ANALYSIS_reto_grande_garage_extra_features


...

,model,collections,mae,mse,rmse
0,all_collections_indoor_garage_extra_features_run0_depth3_model4,reto_grande_outdoor,5.660897,1.551621,7.046409
1,all_collections_indoor_garage_extra_features_run0_depth3_model4,reto_grande_indoor,0.067208,0.000386,0.111080
2,all_collections_indoor_garage_extra_features_run0_depth3_model4,reto_grande_garage,0.311344,0.006468,0.454929
3,all_collections_indoor_garage_extra_features_run0_depth3_model4,all_reto_grande,2.178415,0.560246,4.234131
4,all_collections_indoor_garage_extra_features_run1_depth0_model2,reto_grande_outdoor,5.654307,1.507593,6.945717
5,all_collections_indoor_garage_extra_features_run1_depth0_model2,reto_grande_indoor,0.249348,0.004129,0.363476
6,all_collections_indoor_garage_extra_features_run1_depth0_model2,reto_grande_garage,0.684490,0.027015,0.929769
7,all_collections_indoor_garage_extra_features_run1_depth0_model2,all_reto_grande,2.372153,0.553811,4.209747
8,all_collections_indoor_garage_extra_features_run2_depth0_model3,reto_grande_outdoor,5.302765,1.373570,6.629798
9,all_collections_indoor_garage_extra_features_run2_depth0_model3,reto_grande_indoor,0.511147,0.017999,0.758934


GROUP : CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features


,model,collections,mae,mse,rmse
0,all_collections_garage_outdoor_extra_features_run0_depth0_model3,reto_grande_outdoor,1.236526,0.132349,2.057950
1,all_collections_garage_outdoor_extra_features_run0_depth0_model3,reto_grande_indoor,17.592349,11.506430,19.188688
2,all_collections_garage_outdoor_extra_features_run0_depth0_model3,reto_grande_garage,1.518065,0.188747,2.457621
3,all_collections_garage_outdoor_extra_features_run0_depth0_model3,all_reto_grande,5.033582,2.714931,9.320826
4,all_collections_garage_outdoor_extra_features_run1_depth4_model3,reto_grande_outdoor,0.316600,0.005985,0.437633
5,all_collections_garage_outdoor_extra_features_run1_depth4_model3,reto_grande_indoor,16.104764,13.704412,20.941375
6,all_collections_garage_outdoor_extra_features_run1_depth4_model3,reto_grande_garage,0.423093,0.009594,0.554076
7,all_collections_garage_outdoor_extra_features_run1_depth4_model3,all_reto_grande,3.913165,3.089593,9.943188
8,all_collections_garage_outdoor_extra_features_run2_depth4_model3,reto_grande_outdoor,0.347436,0.013875,0.666336
9,all_collections_garage_outdoor_extra_features_run2_depth4_model3,reto_grande_indoor,7.260839,2.719784,9.329153


GROUP : CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features


,model,collections,mae,mse,rmse
0,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,reto_grande_outdoor,0.106963,0.000751,0.155052
1,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,reto_grande_indoor,0.029014,0.000052,0.040980
2,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,reto_grande_garage,0.115005,0.000822,0.162224
3,all_collections_garage_outdoor_indoor_extra_features_run0_depth3_model0,all_reto_grande,0.092768,0.000624,0.141265
4,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,reto_grande_outdoor,0.135375,0.000961,0.175366
5,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,reto_grande_indoor,0.034469,0.000064,0.045170
6,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,reto_grande_garage,0.146620,0.001227,0.198143
7,all_collections_garage_outdoor_indoor_extra_features_run1_depth0_model5,all_reto_grande,0.117346,0.000870,0.166822
8,all_collections_garage_outdoor_indoor_extra_features_run2_depth1_model5,reto_grande_outdoor,0.403666,0.009060,0.538453
9,all_collections_garage_outdoor_indoor_extra_features_run2_depth1_model5,reto_grande_indoor,0.202010,0.002119,0.260426


In [39]:
import re
import pandas as pd
import matplotlib.pyplot as plt


combined_rows = []

for group, rows in results.items():
    
    dataset = max((d for d in databases if d in group), key=len, default=None)
    space = max((s for s in spaces if s in group), key=len, default=None)
    
    for current_row in rows:
        print(current_row)
        (model_output_name, collections_name, mae_final, mse_final, rmse_final) = current_row
        combined_rows.append((group,collections_name,mae_final,mse_final,rmse_final))

print(combined_rows)

('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_garage', 6.2791118243467405, 1.94230494597081, 7.883765488081562)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_outdoor', 5.898936339339098, 1.9039598711592138, 7.805556730758854)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_indoor', 22.917394976145037, 30.7811758131845, 31.384671832311774)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_garage_outdoor', 6.0204576729081545, 1.9162220187216885, 7.830651607567153)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_garage_indoor', 10.75262514254114, 9.696168238500832, 17.61469226617447)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_indoor_outdoor', 8.405772130035633, 6.157597572662599, 14.037204932792111)
('reto_grande_outdoor_extra_features_run0_depth0_model4', 'all_collections_garage_outdoor_indoor', 7.797553858713284, 4.95203

In [54]:
import pandas as pd

singles = {
    "CNN_DATA_ANALYSIS_reto_grande_garage_exponential"          : "reto_grande_garage_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_exponential"          : "reto_grande_indoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_exponential"         : "reto_grande_outdoor_exponential",
    "CNN_DATA_ANALYSIS_reto_grande_garage_raw"                  : "reto_grande_garage_raw",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_raw"                  : "reto_grande_indoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_raw"                 : "reto_grande_outdoor_raw",
    "CNN_DATA_ANALYSIS_reto_grande_garage_meters"               : "reto_grande_garage_meters",
    "CNN_DATA_ANALYSIS_reto_grande_indoor_meters"               : "reto_grande_indoor_meters",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_meters"              : "reto_grande_outdoor_meters",
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande"                  : "all_spaces_reto_grande",
    "CNN_DATA_ANALYSIS_reto_grande_outdoor_extra_features"      : "reto_grande_outdoor_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_indoor_extra_features"       : "reto_grande_indoor_extra_features", 
    "CNN_DATA_ANALYSIS_reto_grande_garage_extra_features"       : "reto_grande_garage_extra_features", 
    "CNN_DATA_ANALYSIS_all_spaces_reto_grande_extra_features"   : "all_spaces_reto_grande_extra_features", 
}

spaces = {
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor"                      : "all_coll_garage_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage"                       : "all_coll_indoor_garage",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor"                      : "all_coll_indoor_outdoor",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor"               : "all_coll_garage_outdoor_indoor",
    "CNN_DATA_ANALYSIS_all_collections_indoor_outdoor_extra_features"       : "all_coll_indoor_outdoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_indoor_garage_extra_features"        : "all_coll_indoor_garage_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_extra_features"       : "all_coll_garage_outdoor_extra_features",
    "CNN_DATA_ANALYSIS_all_collections_garage_outdoor_indoor_extra_features": "all_coll_garage_outdoor_indoor_extra_features",
}

spaces_datasets = []
singles_datasets = []

for row in combined_rows:
    (group,collections_name,mae_final,mse_final,rmse_final) = row
    if group in spaces:
        group_defnition = spaces[group]
        spaces_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    elif group in singles:
        group_defnition = singles[group]
        singles_datasets.append((group_defnition,collections_name,mae_final,mse_final,rmse_final))
    else:
        print(f"ERROR : {group}")

print(spaces_datasets)
print(singles_datasets)


[('all_coll_indoor_outdoor_extra_features', 'reto_grande_outdoor', 0.07440566927270816, 0.00031111206737277874, 0.09977768365686247), ('all_coll_indoor_outdoor_extra_features', 'reto_grande_indoor', 0.07596758029667915, 0.00026964527344629187, 0.09289052023904991), ('all_coll_indoor_outdoor_extra_features', 'reto_grande_garage', 4.848914759264801, 1.0863808324654145, 5.896116233495848), ('all_coll_indoor_outdoor_extra_features', 'all_reto_grande', 2.059617025295657, 0.451802904076375, 3.8023273044865564), ('all_coll_indoor_outdoor_extra_features', 'reto_grande_outdoor', 0.213040030119974, 0.0026044417583547972, 0.2886903813211544), ('all_coll_indoor_outdoor_extra_features', 'reto_grande_indoor', 0.08585882858312141, 0.00048483433532242175, 0.1245580135130514), ('all_coll_indoor_outdoor_extra_features', 'reto_grande_garage', 4.395630177239051, 0.9654715941779773, 5.558335273595438), ('all_coll_indoor_outdoor_extra_features', 'all_reto_grande', 1.9232116824549088, 0.40241080590451517, 3.

In [55]:
import pandas as pd

# rows: list[tuple] like (dataset, collections, mae, mse, rmse)
def make_wide_by_collection(rows, *,
                            caption="Modelos - erros médios por coleção",
                            label="tab:model_metrics_by_collection"):
    # Build DataFrame
    df = pd.DataFrame(rows, columns=["dataset", "collections", "mae", "mse", "rmse"])

    if df.empty:
        print("No data provided.")
        return None, None, None

    # Robust pivot (handles duplicate (dataset, collections) by averaging)
    df_wide = (
        df.pivot_table(index="dataset",
                       columns="collections",
                       values=["mae", "mse", "rmse"],
                       aggfunc="mean")
          .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
          .sort_index(axis=1, level=[0, 1])        # nice column order
          .sort_index(axis=0)                      # sort datasets
    )

    # Optional: round displayed values only
    styled = (
        df_wide.style
              .format(precision=2, decimal=",")
              .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=caption,
        label=label,
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
    )

    return df, df_wide, (styled, latex.replace("_", "\\_"))


def latex_with_lines(df, *args, **kwargs):
    kwargs['column_format'] = '|'.join([''] + ['l'] * df.index.nlevels
                                            + ['r'] * df.shape[1] + [''])
    res = df.to_latex(*args, **kwargs)
    return res.replace('\\\\\n', '\\\\ \\midrule\n')

In [56]:

# Build DataFrame
df = pd.DataFrame(spaces_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])

# Robust pivot (handles duplicate (dataset, collections) by averaging)
df_wide = (
    df.pivot_table(index="dataset",
                    columns="collections",
                    values=["mae", "mse", "rmse"],
                    aggfunc="mean")
        .swaplevel(0, 1, axis=1)                 # put collection on top, metrics underneath
        .sort_index(axis=1, level=[0, 1])        # nice column order
        .sort_index(axis=0)                      # sort datasets
)

df_wide = df_wide.sort_values(by="dataset", key=lambda x: x.str.len())
df_wide = df_wide.transpose()

# Optional: round displayed values only
styled = (
    df_wide.style
            .format(precision=2, decimal=",")
            .background_gradient(cmap="RdYlGn_r")
)

latex = styled.to_latex(
    caption="Dataset Combination errors",
    label="tb:model_metrics_by_collection_spaces",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
    hrules=True,
)


display(styled)
print(latex.replace("_", "\\_"))




\begin{table}[H]
\caption{Dataset Combination errors}
\label{tb:model\_metrics\_by\_collection\_spaces}
\begin{tabular}{llrrrr}
\toprule
 & dataset & all\_coll\_indoor\_garage\_extra\_features & all\_coll\_garage\_outdoor\_extra\_features & all\_coll\_indoor\_outdoor\_extra\_features & all\_coll\_garage\_outdoor\_indoor\_extra\_features \\
collections &  &  &  &  &  \\
\midrule
\multirow[c]{3}{*}{all\_reto\_grande} & mae & {\cellcolor[HTML]{BFE47A}} \color[HTML]{000000} 2,42 & {\cellcolor[HTML]{70C164}} \color[HTML]{000000} 3,64 & {\cellcolor[HTML]{C1E57B}} \color[HTML]{000000} 2,13 & {\cellcolor[HTML]{FED481}} \color[HTML]{000000} 0,20 \\
 & mse & {\cellcolor[HTML]{148E4B}} \color[HTML]{F1F1F1} 0,56 & {\cellcolor[HTML]{2DA155}} \color[HTML]{F1F1F1} 2,15 & {\cellcolor[HTML]{148E4B}} \color[HTML]{F1F1F1} 0,47 & {\cellcolor[HTML]{026C39}} \color[HTML]{F1F1F1} 0,00 \\
 & rmse & {\cellcolor[HTML]{FEDA86}} \color[HTML]{000000} 4,22 & {\cellcolor[HTML]{F7FCB4}} \color[HTML]{000000} 7,92 & {\

In [64]:
# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["model", "collections", "mae", "mse", "rmse"])

# Map collections labels
CNN_space_combinations = {
    "all_collections_garage"                : "garage",
    "all_collections_outdoor"               : "outdoor",
    "all_collections_indoor"                : "indoor",
    "all_collections_garage_outdoor"        : "garage_outdoor",
    "all_collections_garage_indoor"         : "garage_indoor",
    "all_collections_indoor_outdoor"        : "indoor_outdoor",
    "all_collections_garage_outdoor_indoor" : "garage_outdoor_indoor",
}
df["collections"] = df["collections"].map(CNN_space_combinations)

# Ensure numeric cols are floats (in case they came as strings with commas)
num_cols = ["mae", "mse", "rmse"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c].astype(str).str.replace(",", ".", regex=False), errors="coerce")

# Aggregate and create a MultiIndex (dataset -> collections)
df = (
    df.groupby(["model", "collections"], as_index=False)[num_cols]
      .mean()
      .sort_values(["model", "collections"])
      .set_index(["model", "collections"])   # <- multirow index for rows
)

# Styled display in notebook (keeps the MultiIndex)
styled = (
    df.style
      .format(precision=2, decimal=",")
      .background_gradient(cmap="RdYlGn_r", subset=num_cols)
)

display(styled)

latex = styled.to_latex(
    caption=f"{dataset} error",
    label=f"tb:cnn_all_datasets_{dataset}",
    position="H",
    environment="table",
    convert_css=True,
    multicol_align="c",
    hrules=True,
    # multirow=True,  # uncomment if your pandas version exposes this arg on Styler.to_latex
)

print(latex.replace("_", "\\_"))


\begin{table}[H]
\caption{all\_collections\_garage\_outdoor\_indoor error}
\label{tb:cnn\_all\_datasets\_all\_collections\_garage\_outdoor\_indoor}
\begin{tabular}{llrrr}
\toprule
 &  & mae & mse & rmse \\
model & collections &  &  &  \\
\midrule
\multirow[c]{7}{*}{all\_spaces\_reto\_grande\_extra\_features} & garage & {\cellcolor[HTML]{0E8245}} \color[HTML]{F1F1F1} 5,44 & {\cellcolor[HTML]{06733D}} \color[HTML]{F1F1F1} 1,82 & {\cellcolor[HTML]{0F8446}} \color[HTML]{F1F1F1} 7,52 \\
 & garage\_indoor & {\cellcolor[HTML]{30A356}} \color[HTML]{F1F1F1} 6,89 & {\cellcolor[HTML]{18954F}} \color[HTML]{F1F1F1} 3,65 & {\cellcolor[HTML]{66BD63}} \color[HTML]{F1F1F1} 10,78 \\
 & garage\_outdoor & {\cellcolor[HTML]{17934E}} \color[HTML]{F1F1F1} 6,14 & {\cellcolor[HTML]{097940}} \color[HTML]{F1F1F1} 2,14 & {\cellcolor[HTML]{18954F}} \color[HTML]{F1F1F1} 8,27 \\
 & garage\_outdoor\_indoor & {\cellcolor[HTML]{279F53}} \color[HTML]{F1F1F1} 6,63 & {\cellcolor[HTML]{108647}} \color[HTML]{F1F1F1} 2,83 & 

In [45]:



# Build DataFrame
df = pd.DataFrame(singles_datasets, columns=["dataset", "collections", "mae", "mse", "rmse"])
display(df)

datasets = list(df["collections"].unique())
for dataset in datasets:
    print(f"Dataset: {dataset}")
    df_current = df.loc[df['collections'] == dataset]

    num_cols = ['mae', 'mse', 'rmse']
    df_current = (
        df_current.groupby('dataset', as_index=False)[num_cols]
        .mean()
    )
    df_current = df_current.sort_values('dataset')
    styled = (
        df_current.style
                .format(precision=2, decimal=",")
                .background_gradient(cmap="RdYlGn_r")
    )

    latex = styled.to_latex(
        caption=f"{dataset} error",
        label=f"tb:cnn_all_datasets_{dataset}",
        position="H",
        environment="table",
        convert_css=True,
        multicol_align="c",
        hrules=True,
    )
    

    display(styled)
    print(latex.replace("_", "\\_"))




,dataset,collections,mae,mse,rmse
0,reto_grande_outdoor_extra_features,all_collections_garage,6.279112,1.942305,7.883765
1,reto_grande_outdoor_extra_features,all_collections_outdoor,5.898936,1.903960,7.805557
2,reto_grande_outdoor_extra_features,all_collections_indoor,22.917395,30.781176,31.384672
3,reto_grande_outdoor_extra_features,all_collections_garage_outdoor,6.020458,1.916222,7.830652
4,reto_grande_outdoor_extra_features,all_collections_garage_indoor,10.752625,9.696168,17.614692
...,...,...,...,...,...
79,all_spaces_reto_grande_extra_features,all_collections_indoor,9.868624,6.950116,14.913205
80,all_spaces_reto_grande_extra_features,all_collections_garage_outdoor,5.949038,1.792493,7.573623
81,all_spaces_reto_grande_extra_features,all_collections_garage_indoor,6.733191,3.037265,9.858625
82,all_spaces_reto_grande_extra_features,all_collections_indoor_outdoor,6.674151,2.629982,9.173844


Dataset: all_collections_garage


,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"5,44","1,82","7,52"
1,reto_grande_garage_extra_features,"4,35","1,15","6,05"
2,reto_grande_indoor_extra_features,"9,55","4,25","11,66"
3,reto_grande_outdoor_extra_features,"8,66","3,75","10,72"


\begin{table}[H]
\caption{all\_collections\_garage error}
\label{tb:cnn\_all\_datasets\_all\_collections\_garage}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{6BBF64}} \color[HTML]{000000} 5,44 & {\cellcolor[HTML]{70C164}} \color[HTML]{000000} 1,82 & {\cellcolor[HTML]{8CCD67}} \color[HTML]{000000} 7,52 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 4,35 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 1,15 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 6,05 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,55 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 4,25 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 11,66 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{EB5A3A}} \color[HTML]{F1F1F1} 8,66 & {\cellcolor[HTML]{E95538}} \color[HTML]{F1F1F1} 3,75 & {\cellcolor[HTML]{EA5

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"6,46","2,30","8,55"
1,reto_grande_garage_extra_features,"7,28","2,64","9,18"
2,reto_grande_indoor_extra_features,"9,53","4,23","11,63"
3,reto_grande_outdoor_extra_features,"7,47","2,94","9,61"


...

\begin{table}[H]
\caption{all\_collections\_garage\_indoor error}
\label{tb:cnn\_all\_datasets\_all\_collections\_garage\_indoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 6,89 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 3,65 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 10,78 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{63BC62}} \color[HTML]{F1F1F1} 8,05 & {\cellcolor[HTML]{5DB961}} \color[HTML]{F1F1F1} 4,86 & {\cellcolor[HTML]{7AC665}} \color[HTML]{000000} 12,46 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{EBF7A3}} \color[HTML]{000000} 9,49 & {\cellcolor[HTML]{45AD5B}} \color[HTML]{F1F1F1} 4,67 & {\cellcolor[HTML]{66BD63}} \color[HTML]{F1F1F1} 12,23 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 12,73 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 10,11 & {

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"7,11","3,23","10,11"
1,reto_grande_garage_extra_features,"8,88","4,45","11,93"
2,reto_grande_indoor_extra_features,"9,50","4,46","11,95"
3,reto_grande_outdoor_extra_features,"9,87","6,55","14,45"


\begin{table}[H]
\caption{all\_collections\_indoor\_outdoor error}
\label{tb:cnn\_all\_datasets\_all\_collections\_indoor\_outdoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 7,11 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 3,23 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 10,11 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{FECC7B}} \color[HTML]{000000} 8,88 & {\cellcolor[HTML]{C9E881}} \color[HTML]{000000} 4,45 & {\cellcolor[HTML]{E0F295}} \color[HTML]{000000} 11,93 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{E14430}} \color[HTML]{F1F1F1} 9,50 & {\cellcolor[HTML]{CBE982}} \color[HTML]{000000} 4,46 & {\cellcolor[HTML]{E2F397}} \color[HTML]{000000} 11,95 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,87 & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 6,55 & {

,dataset,mae,mse,rmse
0,all_spaces_reto_grande_extra_features,"6,63","2,83","9,49"
1,reto_grande_garage_extra_features,"7,58","3,51","10,59"
2,reto_grande_indoor_extra_features,"9,52","4,40","11,87"
3,reto_grande_outdoor_extra_features,"9,53","5,75","13,54"


\begin{table}[H]
\caption{all\_collections\_garage\_outdoor\_indoor error}
\label{tb:cnn\_all\_datasets\_all\_collections\_garage\_outdoor\_indoor}
\begin{tabular}{llrrr}
\toprule
 & dataset & mae & mse & rmse \\
\midrule
0 & all\_spaces\_reto\_grande\_extra\_features & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 6,63 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 2,83 & {\cellcolor[HTML]{006837}} \color[HTML]{F1F1F1} 9,49 \\
1 & reto\_grande\_garage\_extra\_features & {\cellcolor[HTML]{B3DF72}} \color[HTML]{000000} 7,58 & {\cellcolor[HTML]{7AC665}} \color[HTML]{000000} 3,51 & {\cellcolor[HTML]{93D168}} \color[HTML]{000000} 10,59 \\
2 & reto\_grande\_indoor\_extra\_features & {\cellcolor[HTML]{A70226}} \color[HTML]{F1F1F1} 9,52 & {\cellcolor[HTML]{FFF2AA}} \color[HTML]{000000} 4,40 & {\cellcolor[HTML]{FEE491}} \color[HTML]{000000} 11,87 \\
3 & reto\_grande\_outdoor\_extra\_features & {\cellcolor[HTML]{A50026}} \color[HTML]{F1F1F1} 9,53 & {\cellcolor[HTML]{A50026}} \color[HTML]{F